In [1]:
import os

In [2]:
%pwd

'c:\\Users\\aarav\\Downloads\\Second-Hand-Car-Price-Prediction-Using-MLops\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\aarav\\Downloads\\Second-Hand-Car-Price-Prediction-Using-MLops'

In [5]:
import pandas as pd

In [32]:
df = pd.read_csv('artifacts/data_ingestion/CleanedCar_main.csv')

In [33]:
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel


In [34]:
df.isnull().sum()

name          0
company       0
year          0
Price         0
kms_driven    0
fuel_type     0
dtype: int64

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815 entries, 0 to 814
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        815 non-null    object
 1   company     815 non-null    object
 2   year        815 non-null    int64 
 3   Price       815 non-null    int64 
 4   kms_driven  815 non-null    int64 
 5   fuel_type   815 non-null    object
dtypes: int64(3), object(3)
memory usage: 38.3+ KB


In [ ]:
## Entity 

In [37]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str 
    unzip_data_dir: Path 
    all_schema: dict 

In [13]:
# Configuration Manager

In [38]:
from CarPrediction.constants import *
from CarPrediction.utils.common import read_yaml,create_directories 

In [39]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
          config = self.config.data_validation 
          schema = self.schema.COLUMNS

          create_directories([config.root_dir])

          data_validation_config = DataValidationConfig(
                root_dir=config.root_dir,
                STATUS_FILE=config.STATUS_FILE,
                unzip_data_dir= config.unzip_data_dir,
                all_schema=schema,
          )
          
          return data_validation_config

In [40]:
# Components

In [41]:
import os
from CarPrediction import logger  

In [42]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config 

    
    def validate_all_columns(self)-> bool:
        try:
            validation_status: None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True 
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e 

In [44]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e  

[2024-01-09 00:59:36,943: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-09 00:59:36,945: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-09 00:59:36,947: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-09 00:59:36,949: INFO: common: created directory at: artifacts]
[2024-01-09 00:59:36,951: INFO: common: created directory at: artifacts/data_validation]
